# 2D Distance Finder
**NOTE:** Currently this tool does not work on EIT, it can only be ran locally! A MATLAB version of this tool is available in the `MATLAB` folder.

This tool allows you to measure the distance between two points on a 2D image. Upload a folder containing the image stack you want to analyze, and click on the image to measure the distance between two points. The distance will be displayed in pixels and millimeters.

<center>
    <img src="https://raw.githubusercontent.com/agadin/QP2_big_data_project_tools/refs/heads/main/img/2D_distance_demo.gif" alt="2D Distance Demo" width="50%" />
</center>


# Upload the Image Stack
Select the folder containing the image stack you want to analyze. The image stack should be in a folder containing the image files (e.g. MRI_4, CT_1, etc.).

In [ ]:
!pip install ipywidgets ipython > /dev/null 2>&1

current_path = 'insert_path_here' # Change this to the path of the folder containing the image stack (ie MRI_1)


# Start the Viewer
Run the cell below to start the viewer. A window will appear with a scroll bar to navigate through the images. Click on the image to measure the distance between two points. The distance will be displayed in pixels and millimeters. There can only be one line present at a time and its distance will be printed to the console.

In [ ]:
import sys
import os
import math
import datetime
import numpy as np
from PyQt5.QtWidgets import QApplication, QMainWindow, QLabel, QVBoxLayout, QWidget, QSlider, QFileDialog
from PyQt5.QtGui import QPixmap, QImage, QPainter, QPen
from PyQt5.QtCore import Qt, QPoint
from PIL import Image


class ImageMeasureApp(QMainWindow):
    def __init__(self, folder_path):
        super().__init__()
        self.folder_path = folder_path
        self.tif_files = self.get_tif_files(folder_path)
        self.current_index = 0
        self.init_ui()

    def get_tif_files(self, folder_path):
        """Retrieve all .tif files in the specified folder."""
        return sorted([f for f in os.listdir(folder_path) if f.endswith(".tif")])

    def init_ui(self):
        if not self.tif_files:
            print(f"No .tif files found in {self.folder_path}")
            sys.exit()

        self.load_image(self.tif_files[self.current_index])

        self.label = QLabel(self)
        self.label.setPixmap(self.pixmap)
        self.label.setAlignment(Qt.AlignTop | Qt.AlignLeft)
        self.label.setScaledContents(True)
        self.label.mousePressEvent = self.get_point

        self.points = []

        self.slider = QSlider(Qt.Horizontal)
        self.slider.setMinimum(0)
        self.slider.setMaximum(len(self.tif_files) - 1)
        self.slider.setValue(self.current_index)
        self.slider.valueChanged.connect(self.update_image)

        self.layout = QVBoxLayout()
        self.layout.addWidget(self.label)
        self.layout.addWidget(self.slider)

        container = QWidget()
        container.setLayout(self.layout)
        self.setCentralWidget(container)

        self.setWindowTitle("Measure Distance on Image with File Navigation")
        self.resize(800, 800)
        self.show()

    def load_image(self, filename):
        """Load the image and prepare the pixmap."""
        image_path = os.path.join(self.folder_path, filename)
        try:
            self.image = Image.open(image_path)
            self.image_array = np.array(self.image)

            height, width = self.image_array.shape
            image_qt = QImage(
                self.image_array.data,
                width,
                height,
                self.image_array.strides[0],
                QImage.Format_Grayscale8
            )
            self.pixmap = QPixmap.fromImage(image_qt)
            self.original_pixmap = self.pixmap.copy()

        except Exception as e:
            print(f"Failed to load image {filename} at {image_path}: {e}")


    def update_image(self, value):
        """Update the displayed image when the slider value changes."""
        self.current_index = value
        self.load_image(self.tif_files[self.current_index])
        self.label.setPixmap(self.pixmap)
        self.points.clear()

    def get_point(self, event):
        """Capture click points on the image and calculate distance."""
        x = event.pos().x()
        y = event.pos().y()
        self.points.append(QPoint(x, y))

        if len(self.points) == 2:
            self.pixmap = self.original_pixmap.copy()
            painter = QPainter(self.pixmap)
            pen = QPen(Qt.blue, 3)
            painter.setPen(pen)
            painter.drawLine(self.points[0], self.points[1])
            painter.end()

            self.label.setPixmap(self.pixmap)
            x1, y1 = self.points[0].x(), self.points[0].y()
            x2, y2 = self.points[1].x(), self.points[1].y()
            pixel_distance = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
            mm_distance = pixel_distance * (10 / 17.52)

            print(f"Distance: {pixel_distance:.2f} pixels ({mm_distance:.2f} mm)")
            self.points.clear()


# Main execution
if __name__ == "__main__":
    app = QApplication(sys.argv)

    window = ImageMeasureApp(current_path)
    now = datetime.datetime.now()
    print(f"{now} - The cell has been completed")
    sys.exit(app.exec_())

## If Continual Error persist about "cannot process image file" use this code

In [ ]:
import sys
import os
import math
import datetime
import numpy as np
import tifffile as tiff
from PyQt5.QtWidgets import QApplication, QMainWindow, QLabel, QVBoxLayout, QWidget, QSlider
from PyQt5.QtGui import QPixmap, QImage, QPainter, QPen
from PyQt5.QtCore import Qt, QPoint

class ImageMeasureApp(QMainWindow):
    def __init__(self, folder_path):
        super().__init__()
        self.folder_path = folder_path
        self.tif_files = self.get_tif_files(folder_path)
        self.current_index = 0
        self.points = []
        self.pixmap = None  # Avoid attribute error
        self.init_ui()

    def get_tif_files(self, folder_path):
        """Retrieve all .tif files in the specified folder."""
        return sorted([f for f in os.listdir(folder_path) if f.lower().endswith(".tif")])

    def init_ui(self):
        if not self.tif_files:
            print(f"No .tif files found in {self.folder_path}")
            sys.exit()

        self.label = QLabel(self)
        self.load_image(self.tif_files[self.current_index])
        if self.pixmap:
            self.label.setPixmap(self.pixmap)
        else:
            self.label.setText("Failed to load image")

        self.label.setAlignment(Qt.AlignTop | Qt.AlignLeft)
        self.label.setScaledContents(True)
        self.label.mousePressEvent = self.get_point

        self.slider = QSlider(Qt.Horizontal)
        self.slider.setMinimum(0)
        self.slider.setMaximum(len(self.tif_files) - 1)
        self.slider.setValue(self.current_index)
        self.slider.valueChanged.connect(self.update_image)

        layout = QVBoxLayout()
        layout.addWidget(self.label)
        layout.addWidget(self.slider)

        container = QWidget()
        container.setLayout(layout)
        self.setCentralWidget(container)

        self.setWindowTitle("Measure Distance on Image")
        self.resize(800, 800)
        self.show()

    def load_image(self, filename):
        """Load the image using tifffile and convert it to QPixmap."""
        image_path = os.path.join(self.folder_path, filename)
        try:
            image = tiff.imread(image_path)  # Read TIFF image

            # Convert high-bit-depth images to 8-bit grayscale for display
            if image.dtype == np.uint64 or image.dtype == np.int64 or image.dtype == np.float64:
                image = (image / np.max(image) * 255).astype(np.uint8)

            # If image is multi-channel, convert to grayscale
            if len(image.shape) > 2:
                image = np.mean(image, axis=2).astype(np.uint8)

            height, width = image.shape
            bytes_per_line = width
            q_image = QImage(image.data, width, height, bytes_per_line, QImage.Format_Grayscale8)
            self.pixmap = QPixmap.fromImage(q_image)
            self.original_pixmap = self.pixmap.copy()

        except Exception as e:
            print(f"Failed to load image {filename} at {image_path}: {e}")
            self.pixmap = None

    def update_image(self, value):
        """Update the displayed image when the slider value changes."""
        self.current_index = value
        self.load_image(self.tif_files[self.current_index])
        if self.pixmap:
            self.label.setPixmap(self.pixmap)
        else:
            self.label.setText("Failed to load image")
        self.points.clear()

    def get_point(self, event):
        """Capture click points on the image and calculate distance."""
        x, y = event.pos().x(), event.pos().y()
        self.points.append(QPoint(x, y))

        if len(self.points) == 2:
            self.pixmap = self.original_pixmap.copy()
            painter = QPainter(self.pixmap)
            pen = QPen(Qt.blue, 3)
            painter.setPen(pen)
            painter.drawLine(self.points[0], self.points[1])
            painter.end()

            self.label.setPixmap(self.pixmap)
            x1, y1 = self.points[0].x(), self.points[0].y()
            x2, y2 = self.points[1].x(), self.points[1].y()
            pixel_distance = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
            mm_distance = pixel_distance * (10 / 17.52)

            print(f"Distance: {pixel_distance:.2f} pixels ({mm_distance:.2f} mm)")
            self.points.clear()


# Main execution
if __name__ == "__main__":
    app = QApplication(sys.argv)

    window = ImageMeasureApp(current_path)
    now = datetime.datetime.now()
    print(f"{now} - The script has started successfully")
    sys.exit(app.exec_())